In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("meowmeowmeowmeowmeow/gtsrb-german-traffic-sign")

print("Path to dataset files:", path)

/Users/ihsanbolum/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Path to dataset files: /Users/ihsanbolum/.cache/kagglehub/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/versions/1


In [2]:
path = "/Users/ihsanbolum/.cache/kagglehub/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/versions/1"
import os
import numpy as np
import cv2
import pandas as pd
from tqdm import tqdm

In [3]:
print(f"files {os.listdir(path)}")

img_height = 32
img_width = 32
num_clases = 43

train_cvs_path = os.path.join(path, "Train.csv")
data = pd.read_csv(train_cvs_path)

print(f"{len(data)} training files")

imgs = []
labels = []

files ['Test', 'Test.csv', 'Meta', 'Train', 'Meta.csv', 'Train.csv']
39209 training files


In [4]:
print(data.shape[0])

39209


In [5]:
for index, row in tqdm(data.iterrows(), total=data.shape[0]):
    img_path = os.path.join(path, row["Path"])
    img = cv2.imread(img_path)
    img = cv2.resize(img, (img_height, img_width))
    img = img / 255.0
    imgs.append(img)
    labels.append(row["ClassId"])

100%|██████████| 39209/39209 [00:12<00:00, 3159.60it/s]


In [6]:
x_train = np.array(imgs)
y_train = np.array(labels)

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

x_train, y_train, y_validation, x_validation = train_test_split(
    x_train,
    y_train,
    test_size=0.2,
    random_state=42,
    stratify=y_train
)

print(f"x_train shape: {x_train.shape}")
print(f"x_validation shape: {x_validation.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_validation shape: {y_validation.shape}")

y_train_catego = to_categorical(y_train, num_clases)
y_validation_catego = to_categorical(y_validation, num_clases)



x_train shape: (31367, 32, 32, 3)
x_validation shape: (7842,)
y_train shape: (7842, 32, 32, 3)
y_validation shape: (31367,)


In [8]:
print(f"origin label {y_train[0]}")
print(f"hot encoded version {y_train_catego[0]}")

origin label [[[0.38431373 0.56862745 0.64705882]
  [0.38823529 0.55294118 0.64705882]
  [0.48627451 0.65882353 0.69803922]
  ...
  [0.21568627 0.30980392 0.34901961]
  [0.23137255 0.31764706 0.33333333]
  [0.21568627 0.28627451 0.30196078]]

 [[0.3372549  0.55686275 0.63137255]
  [0.34509804 0.5254902  0.63921569]
  [0.26666667 0.46666667 0.49019608]
  ...
  [0.21568627 0.33333333 0.34117647]
  [0.25098039 0.35294118 0.35686275]
  [0.24705882 0.29411765 0.29411765]]

 [[0.26666667 0.44705882 0.55686275]
  [0.36470588 0.5254902  0.63529412]
  [0.27843137 0.47843137 0.52941176]
  ...
  [0.21568627 0.30588235 0.35294118]
  [0.24313725 0.30980392 0.30980392]
  [0.26666667 0.3254902  0.34117647]]

 ...

 [[0.23137255 0.30588235 0.3372549 ]
  [0.45490196 0.48627451 0.42745098]
  [0.34509804 0.43529412 0.34509804]
  ...
  [0.11372549 0.16470588 0.58039216]
  [0.29411765 0.25882353 0.59215686]
  [0.37647059 0.40784314 0.58039216]]

 [[0.35294118 0.41568627 0.57647059]
  [0.36078431 0.44313725

In [9]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()

model.add(Conv2D(32, (3,3), activation= "relu", input_shape=(img_height,img_width, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), activation= "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(num_clases, activation="softmax"))

model.compile(
    optimizer = "adam",
    loss= "categorical_crossentropy",
    metrics = ["accuracy"]
)

model.summary()

/Users/ihsanbolum/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 43)             │         5,547 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 319,979 (1.22 MB)

 Trainable params: 319,979 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
train_epochs = 12
history = model.fit(
    x_validation,
    y_train_catego,
    epochs = 12,
    validation_data= (x_validation, y_train_catego),
    batch_size=32
)

print("finito")

Epoch 1/12


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("sequential_1/Cast:0", shape=(None,), dtype=float32). Expected shape (None, 32, 32, 3), but input has incompatible shape (None,)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None,), dtype=int64)
  • training=True
  • mask=None
  • kwargs=<class 'inspect._empty'>